In [1]:
seed = 1
import numpy as np

def seed_torch(seed=1029):
    np.random.seed(seed)
    torch.manual_seed(seed)



In [2]:
import torch.nn as nn
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your RNN
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='tanh')

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # One time step
        # We need to detach the hidden state to prevent exploding/vanishing gradients
        # This is part of truncated backpropagation through time (BPTT)
        out, hn = self.rnn(x, h0.detach())

        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out) 
        # out.size() --> 100, 10
        return out

In [3]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from tqdm import tqdm
import torch
import xlrd
import pandas as pd

book = xlrd.open_workbook('dataset for standardisation.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_unreplicated = [[sheet.cell_value(r, c)
         for c in range(1,9)] for r in range(2,74)]
scaler = StandardScaler()
scaler.fit(data_unreplicated)


book = xlrd.open_workbook('prepareddatakfold.xlsx')
sheet = book.sheet_by_name('Sheet1')
replicated_data = [[sheet.cell_value(r, c)
         for c in range(0,8)] for r in range(0,7200)]

def create_inout_sequences(input_data, tw):
    training_seq = []
    label_seq = []
    L = len(input_data)
    for i in range(0,L,tw):
        train_seq = input_data[i:i+tw,0:5]
        train_label = input_data[i:i+tw,5:8]
        training_seq.append(train_seq)
        label_seq.append(train_label)
    return training_seq,label_seq

standardised_data = scaler.transform(replicated_data)
x = torch.tensor(standardised_data)

z, t = create_inout_sequences(x, 12)
x_train = torch.stack(z)
y_train = torch.stack(t)
n_splits = 6
splits = list(KFold(n_splits=n_splits, shuffle=False, random_state=seed)
              .split(x_train, y_train))




In [4]:
import copy
i = n_splits-1
test_batchsizes = [1,2,3,4,5,6,7,8,9,10,20,40,60,60,100]
test_lrs = [0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,
            0.001,0.002,0.003,0.004,0.005]
input_dim = 5
hidden_dim = 25
layer_dim = 2
output_dim = 3
model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)
init_state = copy.deepcopy(model.state_dict())
for batchsize in test_batchsizes:
    for test_lr in test_lrs:
            loss_fn = torch.nn.MSELoss(reduction='sum')
            learning_rate = test_lr
            av_opt_epoch=0
            av_opt_loss=0
            for i, (train_idx, valid_idx) in enumerate(splits):
                optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
                model.load_state_dict(init_state)
                x_train_fold = torch.tensor(x_train[train_idx], dtype=torch.float32)
                y_train_fold = torch.tensor(y_train[train_idx], dtype=torch.float32)
                x_val_fold = torch.tensor(x_train[valid_idx], dtype=torch.float32)
                y_val_fold = torch.tensor(y_train[valid_idx], dtype=torch.float32)
                train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
                valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)
                train_loader = torch.utils.data.DataLoader(train, batch_size=batchsize,
                                                           shuffle=True)
                valid_loader = torch.utils.data.DataLoader(valid, batch_size=batchsize,
                                                           shuffle=False)
                UP=0
                epoch=0
                k=0
                opt_loss=1000
                old_avg_val_loss=0
                while UP<4 and epoch<300:
                    epoch+=1
                    k+=1
                    model.train()
                    for x_batch, y_batch in train_loader:
                        y_pred = model(x_batch)
                        loss = loss_fn(y_pred, y_batch)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                    model.eval()
                    avg_val_loss = 0.
                    for x_batch, y_batch in valid_loader:
                        y_pred = model(x_batch).detach()
                        avg_val_loss += (loss_fn(y_pred, y_batch).item() / 
                        (len(valid_loader.dataset)*x_batch.size(1)))
                    if k > 4:
                        if avg_val_loss> old_avg_val_loss:
                            UP+=1
                        if avg_val_loss<old_avg_val_loss:
                            UP=0
                        old_avg_val_loss=avg_val_loss
                        k=0
                    if avg_val_loss<opt_loss:
                        opt_loss = avg_val_loss
                        opt_epoch = epoch
                
                av_opt_epoch+=opt_epoch/n_splits
                av_opt_loss+=opt_loss/n_splits
            print(batchsize,test_lr,av_opt_epoch,av_opt_loss)


C:\Users\Gabriel\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Gabriel\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Gabriel\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Gabriel\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().det

1 0.0001 46.5 1.2483054410749015
1 0.0002 30.0 1.2233866854508715
1 0.0003 29.333333333333336 1.1951608682341046
1 0.0004 3.6666666666666665 1.1888165075911412
1 0.0005 8.666666666666668 1.1879972355895572
1 0.0006 4.0 1.194553666975763
1 0.0007 12.166666666666666 1.1937147261699042
1 0.0008 22.499999999999996 1.2212794641653693
1 0.0009 6.166666666666666 1.2668500853909386
1 0.001 54.49999999999999 1.1956374606159001
1 0.002 25.000000000000007 1.140637881424692
1 0.003 17.833333333333336 1.174386572043101
1 0.004 14.333333333333332 1.1000709764162697
1 0.005 31.833333333333332 1.1778374572594958
2 0.0001 64.83333333333333 1.2673262034522164
2 0.0002 29.5 1.2593553339110481
2 0.0003 39.5 1.205446089108785
2 0.0004 34.66666666666667 1.2554481095737882
2 0.0005 13.0 1.194572988483641
2 0.0006 23.666666666666664 1.212956133418613
2 0.0007 36.666666666666664 1.2219095942709182
2 0.0008 26.666666666666668 1.1529667439725664
2 0.0009 22.833333333333332 1.1581780086623297
2 0.001 3.5 1.230361

60 0.003 13.666666666666666 1.2853498119778104
60 0.004 22.333333333333336 1.215313991970486
60 0.005 55.833333333333336 1.2183089065551758
100 0.0001 167.0 1.7598644341362848
100 0.0002 142.99999999999997 1.6668384467230903
100 0.0003 121.33333333333334 1.3494934421115452
100 0.0004 74.00000000000001 1.656617702907986
100 0.0005 87.5 1.2860209062364365
100 0.0006 75.16666666666666 1.274664518568251
100 0.0007 62.50000000000001 1.2724451276991104
100 0.0008 59.666666666666664 1.263226538764106
100 0.0009 54.33333333333333 1.2561819458007812
100 0.001 53.833333333333336 1.2596838972303603
100 0.002 43.5 1.1825265333387587
100 0.003 27.166666666666664 1.2350016530354817
100 0.004 19.166666666666668 1.2692424096001518
100 0.005 39.5 1.29018805609809


In [7]:
torch.save(model.state_dict(), ("C:/Users/Gabriel/Documents/Python/ANN3.pt"))                
                                